In [ ]:
# **Story 7 - Enegy Production**
# **Research Question:**
# *** How do state-level energy production patterns reflect regional strengths and vulnerabilities, and what are the implications for U.S. energy security?***


In [2]:
!pip install pandas openpyxl plotly folium geopandas
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 8.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [folium] 8/11 [geopandas]


In [11]:
prod_df = pd.read_excel('/Users/leslietavarez/Downloads/P5B.xlsx', sheet_name="P5B", skiprows=4)
prod_df.head()

# Extract final set of State + Total Energy columns
cleaned = prod_df[['State.4', 'Trillion Btu.3']].copy()

# Rename columns
cleaned.columns = ['State', 'Total_Energy_Production_2022']

# Drop missing states or non-state rows
cleaned = cleaned.dropna(subset=['State', 'Total_Energy_Production_2022'])

# Remove any rows that aren't actual states (e.g., 'United States', footnotes)
non_states = ['United States', 'United States e', 'Other', 'District of Columbia']
cleaned = cleaned[~cleaned['State'].isin(non_states)]

# Reset index
cleaned = cleaned.reset_index(drop=True)

# Preview cleaned data
cleaned.head()


,State,Total_Energy_Production_2022
0,Texas,25142.391
1,Pennsylvania,9950.508
2,New Mexico,6690.243
3,Wyoming,6013.674
4,West Virginia,5762.856


In [31]:
xls = pd.ExcelFile('/Users/leslietavarez/Downloads/use_tot_realgdp.xlsx')
print(xls.sheet_names)

# Load the relevant sheet
consumption_df = pd.read_excel(xls, sheet_name='Total consumption', skiprows=2)
consumption_df.head()
print(consumption_df.columns)

# Extract just the State and 2022 energy consumption columns
consump_2022 = consumption_df[['State', 2022]].copy()

# Rename columns for clarity
consump_2022.columns = ['State', 'Total_Energy_Consumption_2022']

# Drop rows that are not states (like U.S. total or DC)
non_states = ['United States', 'District of Columbia', 'Other']
consump_2022 = consump_2022[~consump_2022['State'].isin(non_states)]

# Reset index
consump_2022.reset_index(drop=True, inplace=True)

# Preview cleaned data
consump_2022.head()

# Dictionary to convert state abbreviations to full names
!pip install us
import us

# Create a mapping dictionary: 'AL' -> 'Alabama'
abbr_to_name = {state.abbr: state.name for state in us.states.STATES}

# Apply the mapping to your dataframe
consump_2022['State'] = consump_2022['State'].map(abbr_to_name)

# Drop any rows that didn't map properly (e.g., NaN values)
consump_2022 = consump_2022.dropna(subset=['State'])

# Preview cleaned dataframe
consump_2022.head()



['Contents', 'Total consumption', 'Real GDP', 'Energy consumption per real GDP']
Index(['State',    1960,    1961,    1962,    1963,    1964,    1965,    1966,
          1967,    1968,    1969,    1970,    1971,    1972,    1973,    1974,
          1975,    1976,    1977,    1978,    1979,    1980,    1981,    1982,
          1983,    1984,    1985,    1986,    1987,    1988,    1989,    1990,
          1991,    1992,    1993,    1994,    1995,    1996,    1997,    1998,
          1999,    2000,    2001,    2002,    2003,    2004,    2005,    2006,
          2007,    2008,    2009,    2010,    2011,    2012,    2013,    2014,
          2015,    2016,    2017,    2018,    2019,    2020,    2021,    2022],
      dtype='object')
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [us]


,State,Total_Energy_Consumption_2022
0,Alaska,724059
1,Alabama,1902374
2,Arkansas,1052517
3,Arizona,1526882
4,California,6882442


In [39]:
energy_df = pd.merge(cleaned, consump_2022, on='State', how='inner')

# Calculate net production (positive = net exporter, negative = net importer)
energy_df['Net_Production'] = energy_df['Total_Energy_Production_2022'] - energy_df['Total_Energy_Consumption_2022']

# Calculate dependency ratio (consumption / production)
energy_df['Dependency_Ratio'] = energy_df['Total_Energy_Consumption_2022'] / energy_df['Total_Energy_Production_2022']

energy_df.head()


,State,Total_Energy_Production_2022,Total_Energy_Consumption_2022,Net_Production,Dependency_Ratio
0,Texas,25142.391,13780584,-1.375544e+07,548.101571
1,Pennsylvania,9950.508,3736887,-3.726936e+06,375.547359
2,New Mexico,6690.243,687574,-6.808838e+05,102.772650
3,Wyoming,6013.674,496162,-4.901483e+05,82.505636
4,West Virginia,5762.856,835489,-8.297261e+05,144.978289


In [59]:
import folium
import branca.colormap as cm
import requests
import us

# Map full state names to abbreviations
name_to_abbr = {s.name: s.abbr for s in us.states.STATES}
energy_df['State_Abbr'] = energy_df['State'].map(name_to_abbr)

# Load GeoJSON
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json'
geojson = requests.get(geojson_url).json()

# Add popup HTML to GeoJSON features
popup_data = energy_df.set_index('State_Abbr').to_dict('index')
for f in geojson['features']:
    s = f['id']
    info = popup_data.get(s)
    if info:
        f['properties']['popup'] = (
            f"<b>{info['State']}</b><br>"
            f"Production: {info['Total_Energy_Production_2022']:,} Trillion Btu<br>"
            f"Consumption: {info['Total_Energy_Consumption_2022']:,} Trillion Btu<br>"
            f"Net Production: {info['Net_Production']:,} Trillion Btu<br>"
            f"Dependency Ratio: {info['Dependency_Ratio']:.2f}<br>"
            f"Status: {'Net Exporter' if info['Net_Production'] > 0 else 'Net Importer'}"
        )
    else:
        f['properties']['popup'] = f"<b>{s}</b><br>No data"

# Create base map
m = folium.Map(location=[37.8, -96], zoom_start=4, tiles='CartoDB positron')

# Create a linear colormap between min and max dependency ratio
min_ratio = energy_df['Dependency_Ratio'].min()
max_ratio = energy_df['Dependency_Ratio'].max()
colormap = cm.LinearColormap(['green', 'yellow', 'red'], vmin=min_ratio, vmax=max_ratio)
colormap.caption = 'Energy Dependency Ratio (Consumption / Production)'
colormap.add_to(m)

# Function to style each feature based on dependency ratio
def style_function(feature):
    abbr = feature['id']
    val = popup_data.get(abbr, {}).get('Dependency_Ratio', None)
    return {
        'fillColor': colormap(val) if val is not None else 'lightgray',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7,
    }

# Add GeoJson layer with styles and popups
folium.GeoJson(
    geojson,
    style_function=style_function,
    popup=folium.GeoJsonPopup(fields=['popup'], parse_html=True),
    name='Energy Dependency'
).add_to(m)

m



In [61]:
import folium
from folium.features import DivIcon
import geopandas as gpd

# Load GeoJSON into GeoDataFrame for easier centroid extraction
gdf = gpd.GeoDataFrame.from_features(geojson_data["features"])

# Map for state abbr to name (if needed)
abbr_to_name = {state.abbr: state.name for state in us.states.STATES}

# Add labels on the map using state centroids
for feature in geojson_data['features']:
    state_abbr = feature['id']
    # Get centroid of polygon
    geometry = feature['geometry']
    
    # Use shapely to get centroid point coordinates
    from shapely.geometry import shape
    polygon = shape(geometry)
    centroid = polygon.centroid
    lon, lat = centroid.x, centroid.y

    # Add DivIcon label for state abbreviation
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 10pt; font-weight: bold">{state_abbr}</div>',
        )
    ).add_to(m)
m

In [62]:
# **Conclusion:**
# The map shows that most U.S. states have a low energy dependency ratio, indicating that they produce energy at levels close to or exceeding their consumption. This widespread self-sufficiency is especially prominent in energy-rich states such as Texas, Wyoming, and New Mexico, which are net energy exporters. Only a few states, mainly in the Northeast and parts of the West Coast, show higher dependency ratios, relying more heavily on energy imports to meet their needs.

#This general trend toward self-sufficiency enhances energy security across much of the country, reducing vulnerability to external supply shocks. However, the states with higher dependency ratios may still face risks that necessitate targeted policy interventions, such as investing in local energy infrastructure or diversifying energy sources. Overall, the map highlights opportunities for regional cooperation to balance production and consumption while promoting sustainable energy policies tailored to each state’s unique energy profile.